<a href="https://colab.research.google.com/github/Dharvi-k/BHAGAVAD_GITA_CHATBOT/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from getpass import getpass
from huggingface_hub import login

token = getpass("Enter your Hugging Face token: ")



Enter your Hugging Face token: ··········


In [2]:
# setting device agnostic code

import torch

device= 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [3]:
!pip install -r requirements-dita-chatbot.txt    # saved python library and gpu


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements-dita-chatbot.txt'


In [4]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [6]:
"""# upload the files in colab
from google.colab import files
uploaded=files.upload()"""


'# upload the files in colab\nfrom google.colab import files\nuploaded=files.upload()'

In [7]:
# Importing dependencies
import pandas as pd

In [8]:
# Load the files in colab
verses_data=pd.read_json('gita_translation_data.json')
modern_data=pd.read_json('modern_teachings.json')

In [9]:
verses_data = pd.DataFrame(verses_data)
modern_data = pd.DataFrame(modern_data)


In [10]:
# Print first five rows of both the datasets
print('DATASET 1 (GITA VERSES)')
verses_data.head()

DATASET 1 (GITA VERSES)


,verse_id,verse_number,author_id,language_id,text
0,100,1,1,1,"The King Dhritarashtra asked, ""O Sanjaya! What..."
1,101,2,1,1,"Sanjaya replied: ""When Prince Duryodhana saw t..."
2,102,3,1,1,Revered Father! Behold this mighty host of the...
3,103,4,1,1,In it are heroes and great bowmen; the equals ...
4,104,5,1,1,"Dhrishtaketu, Chekitan, the valiant king of Be..."


In [11]:
print('DATASET 2 (MODERN TEACHINGS)')
modern_data.head()

DATASET 2 (MODERN TEACHINGS)


,teaching_id,topic,keywords,text,source_refs
0,1,Identity & True Self,"[identity, soul, self, body, who am I]",The Gita teaches that our true identity is not...,"[{'type': 'verse_commentary', 'reference': 'BG..."
1,2,Equality & Universal Oneness,"[equality, oneness, all beings, samatva]","Though outwardly different, all beings share t...","[{'type': 'verse_commentary', 'reference': 'BG..."
2,3,Duty & Selfless Action (Karma Yoga),"[karma yoga, duty, selfless action, work]",The Gita emphasizes performing duty without at...,"[{'type': 'book', 'reference': 'Tilak – Gita R..."
3,4,Stress Management,"[stress, anxiety, calmness, peace]",By focusing on action and surrendering outcome...,"[{'type': 'verse_commentary', 'reference': 'BG..."
4,5,Work-Life Balance,"[balance, work, life, discipline]","The Gita advises moderation in eating, sleepin...","[{'type': 'verse_commentary', 'reference': 'BG..."


**Check the structure of the Datasets**

In [12]:
# Check the structure of the verses_data dataset
print(verses_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 701 entries, 0 to 700
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   verse_id      701 non-null    int64 
 1   verse_number  701 non-null    int64 
 2   author_id     701 non-null    int64 
 3   language_id   701 non-null    int64 
 4   text          701 non-null    object
dtypes: int64(4), object(1)
memory usage: 27.5+ KB
None


In [13]:
#  Check the structure of the modern_data dataset
print( modern_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   teaching_id  65 non-null     int64 
 1   topic        65 non-null     object
 2   keywords     65 non-null     object
 3   text         65 non-null     object
 4   source_refs  20 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.7+ KB
None


In [14]:
# Length of rows in our datasets
print('Length of verses_data dataset: ',len(verses_data))
print('Length of modern_data dataset: ',len(modern_data))

Length of verses_data dataset:  701
Length of modern_data dataset:  65


In [15]:
# Check for missing values
print('Number of missing values in verses_data dataset: ',verses_data.isnull().sum().sum())
print('Number of missing values in modern_data dataset: ',modern_data.isnull().sum().sum())

Number of missing values in verses_data dataset:  0
Number of missing values in modern_data dataset:  45


*Since there are 45 missing valuesin the column source_refs, let's delete that column*

In [16]:
modern_data.isnull().sum()

,0
teaching_id,0
topic,0
keywords,0
text,0
source_refs,45


In [17]:
modern_data=modern_data.drop('source_refs',axis=1)

In [18]:
# Checking if the column has deleted or not
modern_data.isnull().sum()

,0
teaching_id,0
topic,0
keywords,0
text,0


In [19]:
# CHECK FOR FORMATTING ISSUES
'''
* Remove unwanted spaces

* Ensure all text values are strings

* Convert special characters if needed

'''
# Ensure text column is a string
verses_data['text']=verses_data['text'].astype(str)
modern_data['text']=modern_data['text'].astype(str)

# Removing unwanted spaces at the start or end of a string
verses_data['text']=verses_data['text'].str.strip()
modern_data['text']=modern_data['text'].str.strip()

# Normalize multiple spaces
verses_data['text']=verses_data['text'].str.replace('\s+',' ')
modern_data['text']=modern_data['text'].str.replace('\s+',' ')

<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:20: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:20: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-4090053633.py:19: SyntaxWarning: invalid escape sequence '\s'
  verses_data['text']=verses_data['text'].str.replace('\s+',' ')
/tmp/ipython-input-4090053633.py:20: SyntaxWarning: invalid escape sequence '\s'
  modern_data['text']=modern_data['text'].str.replace('\s+',' ')


In [20]:
# Checking any repeating verses in verses_data dataset
verses_data['text'].value_counts()

,count
text,
No changes needed.,2
"Lord Shri Krishna replied: My beloved friend! It is only through My grace and power that you have been able to see this vision of splendour, the Universal, the Infinite, the Original. Never has it been seen by anyone else.",1
"It is not possible for man on earth to see what thou hast seen, O thou foremost hero of the Kuru-clan, through study of the scriptures, sacrifice or gift, ritual or rigorous austerity!",1
"Do not be afraid or confused by the terrible vision. Put away your fear and, with a joyful mind, see Me again in My usual form.",1
"Sanjaya continued: ""Having thus spoken to Arjuna, Lord Shri Krishna showed Himself again in His accustomed form; and the Mighty Lord, in gentle tones, softly consoled him who had recently trembled with fear.",1
...,...
"Let him seek liberation with the help of his Highest Self, and never disgrace his own Self. For that Self is his only friend; yet it can also be his enemy.",1
"To one who has conquered their lower nature with its help, the Self is a friend; but to one who has not done so, it is an enemy.",1
"The Self of one who is self-controlled and has attained peace is equally unmoved by heat or cold, pleasure or pain, honor or dishonor.",1


In [21]:
# Since in our verses_data dataset we are having 2 placeholder text so delete that
# Remove these entries from the dataset
verses_data=verses_data[~verses_data['text'].str.strip().str.lower().str.replace(r'[^\w\s]', '', regex=True).isin(['','no changes needed'])]

# Also drop rows where text is nan
verses_data=verses_data.dropna(subset=['text'])

print("\nAfter cleaning, total verses left:", len(verses_data))


After cleaning, total verses left: 699


**STEP 2: PRE-PROCESSING**

First five preprocessing steps:

1. Lowercasing

2. Removing punctuation

3. Tokenization

4. Stopword removal

5. Lemmatization

In [22]:
# Importing dependencies
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('punkt_tab')


# 1. Initialize the tools
stop_words=set(stopwords.words('english'))
lemmatizer=WordNetLemmatizer()

# custom stopwords
# keep spiritual words like "thou", "thy", etc
custom_stopwords=["thou", "thy","thee", "shall"]
stop_words.update(custom_stopwords)

# 2. Preparing functions  (Defines a function preprocess_text that takes text as input and returns cleaned tokens.)

def preprocess_text(text):
   # Lowercase
   text=text.lower()

   # Remove punctuation & special characters (keep only letters and spaces)
   text=re.sub(r'[^a-z\s]','',text)

   # Tokenize
   tokens=nltk.word_tokenize(text)

   # Remove stopwords
   tokens=[word for word in tokens if word not in stop_words]

   # Lemmatize
   tokens=[lemmatizer.lemmatize(token) for token in tokens]

   return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [23]:
# Apply preprocessing to both datasets
verses_data['tokens']=verses_data['text'].apply(preprocess_text)
modern_data['tokens']=modern_data['text'].apply(preprocess_text)

In [24]:
verses_data['tokens'].head()

,tokens
0,"[king, dhritarashtra, asked, sanjaya, happened..."
1,"[sanjaya, replied, prince, duryodhana, saw, pa..."
2,"[revered, father, behold, mighty, host, pandav..."
3,"[hero, great, bowman, equal, battle, arjuna, b..."
4,"[dhrishtaketu, chekitan, valiant, king, benare..."


In [25]:
# Install
!pip install sentence-transformers

Here’s what comes next:


1. Take the tokens (or join them into a single string) from each row in both datasets.

2. Pass them to a SentenceTransformer model to get vector embeddings.

3. Store these embeddings for FAISS indexing, which will enable fast similarity search for RAG.

**Preprocessed text → embeddings → FAISS indexes → query retrieval → LLM answer generation.**

**STEP 3: GENERATE EMBEDDINGS**

In [26]:
# Import
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize embedding model (compact, fast, good for semantic similarity)
embedder = SentenceTransformer("all-MiniLM-L6-v2", device="cuda")

# Convert tokens to strings for embedding
verses_data['verses_str'] = verses_data['tokens'].apply(lambda x: ' '.join(x))
modern_data['modern_str'] = modern_data['tokens'].apply(lambda x: ' '.join(x))

# AN INDEXING ERROR IS THERE, SO RESET THE INDEX
verses_data= verses_data.reset_index(drop=True)
modern_data = modern_data.reset_index(drop=True)

# Generate embeddings
verse_embeddings=embedder.encode(verses_data['verses_str'].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True
)
modern_embeddings=embedder.encode(modern_data['modern_str'].tolist(),
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True)

# Check shapes
print("Verse embeddings shape:", verse_embeddings.shape)
print("Modern embeddings shape:", modern_embeddings.shape)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/22 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Verse embeddings shape: (699, 384)
Modern embeddings shape: (65, 384)


***When we build a FAISS index, embeddings must be NumPy arrays of dtype float32.***

In [27]:
# Convert to numpy arrays with float32
verse_embeddings=np.array(verse_embeddings, dtype='float32')
modern_embeddings=np.array(modern_embeddings, dtype='float32')

In [28]:
# INSTALL
!pip install -U langchain-community

In [29]:
!pip install faiss-cpu

In [30]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Combine texts
texts=list(verses_data['verses_str']) + list(modern_data['modern_str'])

# Add metadata
metadatas=(
    [{"type":"verse"} for _ in verses_data['verses_str']]+
    [{"type":"modern"} for _ in modern_data['modern_str']]
)

# Combine embeddings (stacks them row-by-row into one big list (matrix))
all_embeddings=np.vstack([verse_embeddings, modern_embeddings])

# Pair each text with its embedding
text_embeddings=list(zip(texts, all_embeddings))

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Build FAISS directly from texts
vectorstore = FAISS.from_texts(
    texts=texts,
    embedding=embeddings,
    metadatas=metadatas
)

print("Total vectors in index:", vectorstore.index.ntotal)

/tmp/ipython-input-859733070.py:20: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Total vectors in index: 764


*So now we have a vector database (VectorStoreIndex) that contains both datasets (Gita verses + modern explanations) in a unified space.*

**Next Step:**

1. Wrap FAISS in a LangChain Retriever:

In [31]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}       # top 3 results
)

2. Attach Retriever to a LangChain RAG pipeline.

In [32]:
from huggingface_hub import login
login(token)

In [33]:
# 4-bit quantization requires the latest version of bitsandbytes
!pip install -U bitsandbytes
!pip install -U transformers accelerate

In [34]:
import bitsandbytes as bnb
print("bitsandbytes version:", bnb.__version__)


bitsandbytes version: 0.48.1


In [35]:

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain.llms import HuggingFacePipeline

# Load the tokenizer and model
model_name="mistralai/Mistral-7B-Instruct-v0.2"

# Define quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",  # It’s a quantization format used to compress neural network weights from full precision (float32) to 4 bits.
    bnb_4bit_compute_dtype="float16"
)

#Loads the tokenizer associated with Mistral-7B.
#The tokenizer splits your input text into tokens the model understands, and later converts generated tokens back into readable words.
tokenizer=AutoTokenizer.from_pretrained(model_name)

# Loads the actual neural network weights of the Mistral-7B model.
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,           # saves VRAM (Colab friendly)
)

# Create text generation pipeline
pipe=pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
    temperature=0.2,
    return_full_text=False


)

# Wrap in LLM langchain
llm=HuggingFacePipeline(pipeline=pipe)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
/tmp/ipython-input-1036472751.py:39: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipe)


In [36]:
# Check GPU type
!nvidia-smi

Mon Oct  6 04:27:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   75C    P0             32W /   70W |    7184MiB /  15360MiB |      1%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [37]:
# Check python version
!python --version

Python 3.12.11


Then build RetrievalQA

This code is creating a Retrieval-Augmented Generation (RAG) pipeline using LangChain.

- This is a helper function in LangChain that builds a QA system using a retriever and a language model (LLM).

- It automatically wraps the LLM and the retriever into a chain that can handle queries.

In [38]:
from langchain.chains import RetrievalQA

qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

Define intent detection function

In [39]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import re

def clean_query(query):
    # Remove leading/trailing whitespace
    q = query.strip()

    # Remove enclosing quotes
    q = q.strip('"').strip("'")

    # Optional: remove other special characters except basic punctuation
    q = re.sub(r'[^\w\s?.!]', '', q)

    # Lowercase for consistent matching
    q = q.lower()

    return q

# ===============================
# 1️⃣ Summarization chain
# ===============================
summary_prompt = PromptTemplate(
    input_variables=["context"],
    template="Summarize the following Gita teachings in simple words in about 150 words.:\n\n{context}"
)
summarization_chain = LLMChain(llm=llm, prompt=summary_prompt)

# ===============================
# 2️⃣ Intent detection
# ===============================
def detect_intent(query):
    q_lower = query.lower()
    if any(word in q_lower for word in ["summarize", "summary"]):
        return "summary"
    elif any(word in q_lower for word in ["example", "illustration"]):
        return "example"
    elif any(word in q_lower for word in ["explain", "explanation", "meaning of", "what does"]):
        return "explain"
    else:
        return "qa"

# ===============================
# 3️⃣ Sensitive topic detection
# ===============================
def is_sensitive(query):
    sensitive_keywords = [
        "LGBTQ", "politics", "controversy", "war", "fight", "battle",
        "religion", "caste", "society", "mother", "father", "violence",
        "attack", "offensive", "blame","god","gods","gender","sex","religions",
        "kill", "murder", "betray", "marriage", "family", "suicide", "hindu", "muslim"

    ]
    return any(word.lower() in query.lower() for word in sensitive_keywords)

# ===============================
# 4️⃣ Context retrieval
# ===============================
def get_context(query, top_k=3, similarity_threshold=0.6):
    # Get top-k documents with similarity scores from FAISS
    docs_with_scores = retriever.vectorstore.similarity_search_with_score(query, k=top_k)

    # Filter docs based on score threshold
    filtered_docs = [doc for doc, score in docs_with_scores if score >= similarity_threshold]

    if not filtered_docs:
        return "", 0

    # Combine the text of filtered documents
    context_text = "\n".join([doc.page_content for doc in filtered_docs])
    return context_text, len(filtered_docs)


# ===============================
# 5️⃣ Safe handling of sensitive topics
# ===============================
def handle_sensitive(query, context):
  prompt = f"""
  The following user query may be sensitive. You are Vedara, a chatbot specialized in the Bhagavad Gita.
  Answer in a neutral, educational, and respectful way using Bhagavad-Gita teachings.
  Provide context from the Gita in about 150 words. Keep your explanation educational, clear, spiritual, friendly, and easy to understand.
  - Maintain neutrality and avoid personal, political, or religious bias.
  - Approach the sensitive topic thoughtfully and with care.
  - Avoid personal, moral, or lifestyle judgments.
  - If a question touches on modern issues not explicitly addressed in the Gita (like LGBTQ or sexual behavior), respond respectfully and provide general spiritual guidance without bias.

Context from Gita:

  Context from Gita:
  {context}

  User query:
  {query}
  """
  return llm(prompt).strip().replace("\n", " ")

# ----------------------------
# Domain check: only allow Gita-related queries
# ----------------------------
def is_in_domain(query, intent=None):
    # Always allow summary/example/explain queries
    if intent in ["summary", "example", "explain"]:
        return True

    # Keyword-based check
    gita_keywords = ["gita", "krishna", "arjuna", "dharma", "karma", "yoga", "battlefield", "bhagavad", "verse", "teachings", "kurukshetra", "bhagavad gita"]
    if any(word.lower() in query.lower() for word in gita_keywords):
        return True

    return False

# ----------------------------
# Detect simple greetings
# ----------------------------
def is_greeting(query):
    greetings = ["hi", "hello", "hey", "namaste", "good morning", "good evening", "bye", "namaste vedara", "hello vedara", "how are you vedara", "how are you"]
    query_lower = query.lower().strip()
    return query_lower in greetings


def greeting_response():
    return "Namaste, seeker! 🌿📖 I’m Vedara — your playful Gita companion. Ask me anything, and I’ll share the timeless wisdom of the Bhagavad Gita in a heartbeat ⚡✨ Let’s laugh, explore, and discover the eternal truths together 🙏💫."

def generate_explain_prompt(context, query):
    return f"""
    You are Vedara, a friendly Gita chatbot. The user asked you to explain something.
    Provide a clear, educational, and spiritual explanation based on the Bhagavad Gita teachings.
    Keep it simple, around 150 words, and easy to understand.

    Context from Gita:
    {context}

    User query:
    {query}
    """

# ===============================
# 6️⃣ Smart  QA function
# ===============================
def smart_query(query):
    query_clean = clean_query(query)

    # 1️⃣ Greeting check
    if is_greeting(query_clean):
        return greeting_response()

    # 2️⃣ Detect intent
    intent = detect_intent(query_clean)

    # 3️⃣ Domain check: reject out-of-domain queries immediately
    if not is_in_domain(query_clean, intent=intent):
        return "Ah, seeker! 🌞📖 My insights bloom only from the verses of the Bhagavad Gita 🌿✨ Other worlds are beyond my reach 🙏."

    # 4️⃣ Retrieve context
    context, count = get_context(query_clean)

    # 5️⃣ Out-of-context fallback
    if count == 0:
        return "Ah, seeker! 🌞📖 My insights bloom only from the verses of the Bhagavad Gita 🌿✨ Other worlds are beyond my reach 🙏."

    # 6️⃣ Sensitive handling
    if is_sensitive(query_clean):
        return handle_sensitive(query_clean, context)

    # 7️⃣ Answer based on intent
    if intent == "summary":
        return summarization_chain.run(context=context)
    elif intent == "example":
        prompt = f"Provide practical examples based on this Gita context in about 150 words.:\n{context}"
        return llm(prompt).strip().replace("\n", " ")
    elif intent == "explain":
        prompt = generate_explain_prompt(context, query_clean)

    # 8️⃣ Default QA
    prompt = f"""Answer based on Bhagavad Gita teachings in about 150 words.

    Context:
    {context}

    User query:
    {query_clean}
    Do not include labels like "Query:" or "Answer:".
    Keep your response clean, clear, and conversational.
"""
    return llm(prompt).strip().replace("\n", " ")




def extract_paragraph(full_response):
    paragraphs=[p.strip() for p in full_response.split("\n") if p.strip()]
    if paragraphs:
      return paragraphs[-1]
    return full_response.strip()


/tmp/ipython-input-2030155081.py:27: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summarization_chain = LLMChain(llm=llm, prompt=summary_prompt)


In [40]:
# ===============================
# 7️⃣ Example usage
# ===============================
query = "does bhagavad gita promote war"
response = smart_query(query)
print("Response:", response)


/tmp/ipython-input-2030155081.py:95: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return llm(prompt).strip().replace("\n", " ")
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response: Answer:   The Bhagavad Gita, an ancient Indian scripture, is often misunderstood as promoting war due to its setting on a battlefield and Krishna's urging Arjuna to fight. However, the deeper teaching of the Gita is about inner peace, self-control, and standing for truth without hatred or violence. The battlefield symbolizes the inner struggle between right and wrong, and the fight represents the individual's duty (dharma) to uphold truth and righteousness, even in the face of personal gain or cruelty. The Gita's message is not about promoting war but about conquering anger, hatred, and cruelty through compassion, understanding, and nonviolence.


## **Minimal FastAPI wrapper for smart_query**

In [41]:
!pip install pyngrok
from pyngrok import ngrok

In [42]:
!pip install fastapi uvicorn

In [43]:
!ngrok config add-authtoken 32iwkSPOCWqVlKYfjDPp081WQx5_7fZBziZedamaaJGzB4V32

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [44]:
!pip freeze > requirements_gita_chatbot.txt


In [45]:
# list of all active tunnels
tunnels=ngrok.get_tunnels()
print(tunnels)

# disconnect each tunnel
for t in tunnels:
  ngrok.disconnect(t.public_url)

# now start a new tunnel
new_tunnel=ngrok.connect(8000)
print(new_tunnel.public_url)

[]
https://3540706f2c51.ngrok-free.app


In [46]:
ngrok.kill()  # kills any existing tunnels


In [49]:
from fastapi import FastAPI       # create web API endpoints
from pydantic import BaseModel    # define and validate request JSON structure
import uvicorn                    # run the FastAPI server
import nest_asyncio               # allow async server to run inside Jupyter/Colab
from pyngrok import ngrok         # create a public URL to access local server from outside



# FastAPI app
app = FastAPI(title="Bhagavad Gita QA API", version="1.0")

# Request body schema
class QueryRequest(BaseModel):
    query: str

# Root endpoint
@app.get("/")
def home():
    return "Namaste, seeker of wisdom! 🙏 I am Vedara 📖 — your Gita chatbot. My name comes from ‘Veda’ (wisdom) and ‘era’ (time) 🌿, symbolizing ancient knowledge in today’s modern world 🌍. I am here to share the eternal teachings of the Bhagavad Gita, guiding you toward peace and clarity ✨"

# Main query endpoint
@app.post("/ask")
def ask_gita(req: QueryRequest):
    full_response = smart_query(req.query)     # same as Colab
    #answer = extract_paragraph(full_response)  # optional: keep if needed
    return {full_response}                  # JSON response


# GET version (for browser testing)
@app.get("/ask")
def ask_gita_get(query: str):
    full_response = smart_query(query)         # same as Colab
    #answer = extract_paragraph(full_response)  # same processing
    return { full_response}                  # JSON response


#  Run the API in Colab
# lets uvicorn/FastAPI run inside notebooks without crashing due to event loop conflicts.
nest_asyncio.apply()
'''
public_url=ngrok.connect(8000)
print("Public URL:", public_url)

uvicorn.run(app, host="0.0.0.0", port=8000, log_level="debug")'''

# Create public URL
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# ✅ Safe Uvicorn run for notebooks
import asyncio
asyncio.get_event_loop().run_until_complete(
    uvicorn.Server(uvicorn.Config(app, host="0.0.0.0", port=8000))
    .serve()
)


Public URL: NgrokTunnel: "https://69833e314ea7.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [10977]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2401:4900:8811:7be2:a93f:1b89:7465:6206:0 - "GET / HTTP/1.1" 200 OK
INFO:     2401:4900:8811:7be2:a93f:1b89:7465:6206:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2401:4900:8811:7be2:a93f:1b89:7465:6206:0 - "GET /ask?query=hello HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     2401:4900:8811:7be2:a93f:1b89:7465:6206:0 - "GET /ask?query=what%20is%20gita HTTP/1.1" 200 OK


...

INFO:     2401:4900:8811:7be2:a93f:1b89:7465:6206:0 - "GET /ask?query=what%20happened%20in%20kurukshetra HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


INFO:     2401:4900:8811:7be2:a93f:1b89:7465:6206:0 - "GET /ask?query=does%20gita%20support%20hinduism HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [10977]


KeyboardInterrupt: 